# <center> Кластеризация изображений транспортных средств

## Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

from sklearn.cluster import MiniBatchKMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
import pickle

plt.style.use('ggplot')
warnings.simplefilter("ignore")

## 1. Знакомство со структурой данных

In [ ]:
desc_path = 'data/descriptors/'

effnet_data = pickle.load(open(desc_path+'efficientnet-b7.pickle', 'rb')) 
osnet_data = pickle.load(open(desc_path+'osnet.pickle', 'rb'))
vdc_color_data = pickle.load(open(desc_path+'vdc_color.pickle', 'rb'))
vdc_type_data = pickle.load(open(desc_path+'vdc_type.pickle', 'rb'))

effnet_name = 'EfficientNet'
osnet_name = 'OSNet'
vdc_color_name = 'VDC color regression'
vdc_type_name = 'VDC type classification'

additional_message = ''

print_data = lambda name, data: print(f'{name}{additional_message}:\n {data}', '\n', 
                                      f'{data.shape[0]} rows, {data.shape[1]} columns')

In [ ]:
print_data(effnet_name, effnet_data)

In [ ]:
print_data(osnet_name, osnet_data) 

In [ ]:
print_data(vdc_color_name, vdc_color_data)

In [ ]:
print_data(vdc_type_name, vdc_type_data) 

> *Посмотрев на размерности каждой из заданных матриц, можно сказать, что нейросеть EfficientNet описывает изображения наиболее подробным образом - на 2560 дескрипторов. На мой взгляд, в конечном итоге, именно на дескрипторах этой модели кластеризация будет наиболее точной.*

In [ ]:
img_paths = pd.read_csv('data/images_paths.csv')

img_paths.head()

In [ ]:
img_paths['paths'] = ('data/raw_data/' + 
                      img_paths['paths']
                      .apply(lambda x: x.replace('\\', '/')))

img_paths.head()

## 2. Преобразование, очистка и анализ данных

Признаки, найденные с помощью некоторых моделей, исчисляются тысячами, что довольно много. Производить кластеризацию на таком большом количестве признаков, которые были сформированы исходными моделями глубокого обучения, довольно сложно и затратно по времени. 

Понизим размерность исходных дескрипторов с помощью соответствующих методов. Можно уменьшить размерность входных данных до 100 или 200 признаков — этого будет достаточно, чтобы произвести кластеризацию.

In [ ]:
RS = 12 # random_state
additional_message = ' standardised and decomposed'

def standardise_and_decompose(data, n_components):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    
    decomposer = PCA(n_components=n_components, random_state=RS)
    data_decomposed = decomposer.fit_transform(data_scaled)
    
    return data_decomposed

In [ ]:
X_effnet = standardise_and_decompose(effnet_data, n_components=200) 

print_data(effnet_name, X_effnet) 

In [ ]:
X_osnet = standardise_and_decompose(osnet_data, n_components=100)

print_data(osnet_name, X_osnet) 

In [ ]:
X_vdc_color = standardise_and_decompose(vdc_color_data, n_components=50)

print_data(vdc_color_name, X_vdc_color)

In [ ]:
X_vdc_type = standardise_and_decompose(vdc_type_data, n_components=100)

print_data(vdc_type_name, X_vdc_type) 

## 3. Моделирование и оценка качества модели

После предобработки исходных данных произведите кластеризацию для каждого набора дескрипторов.

Для решения задачи используйте несколько различных методов, подобрав оптимальное количество кластеров для каждого метода и варианта дескрипторов.

В качестве метрики для подбора оптимального количества кластеров используйте внутренние меры:
* индекс Калински — Харабаса (`calinski_harabasz_score`) 
* индекс Дэвиса — Болдина (`davies_bouldin_score`)

#### 1. Визуализация результатов кластеризации

In [ ]:
def plot_samples_images(label, data=img_paths, nrows=3, ncols=3):
    """Функция для визуализации нескольких 
       случайных изображений из кластера cluster_label.
       Пути до изображений и метки кластеров должны быть 
       представлены в виде DataFrame со столбцами "paths" и "cluster".

    Args:
        data (DataFrame): таблица с разметкой изображений и соответствующих им кластеров
        cluster_label (int): номер кластера изображений
        nrows (int, optional): количество изображений по строкам таблицы (по умолчанию 3)
        ncols (int, optional): количество изображений по столбцам (по умолчанию 3)
    """
    
    # Фильтруем данные по номеру кластера
    samples_indexes = np.array(data[data['cluster'] == label].index)
    # Перемешиваем результаты
    np.random.shuffle(samples_indexes)
    # Составляем пути до изображений
    paths = data.loc[samples_indexes, 'paths']

    # Создаём фигуру и набор координатных плоскостей
    fig, axes = plt.subplots(nrows, ncols, figsize=(8, 5))
    fig.suptitle(f"Cluster {label}", fontsize=16)
    
    # Создаём циклы по строкам и столбцам в таблице с координатными плоскостями
    for i in range(nrows):
        for j in range(ncols):
            # Определяем индекс пути до изображения
            path_idx = i * ncols + j
            if path_idx >= len(paths):
                break
                
            # Извлекаем путь до изображения
            path = paths.iloc[path_idx]
            
            # Читаем изображение
            img = plt.imread(path)
            
            # Убираем пометки координатных осей
            axes[i,j].axis('off')
            # Отображаем его на соответствующей координатной плоскости
            axes[i,j].imshow(img)


get_sample = lambda X, n=5000: pd.DataFrame(X).sample(n, 
                                                      random_state=RS, 
                                                      ignore_index=False)
"""_summary_"""


def clusters_visualiser(X, y, title, n_components=3, data=img_paths):
    """_summary_

    Args:
        X (_type_): _description_
        y (_type_): _description_
        title (_type_): _description_
        n_components (_type_): _description_

    Returns:
        None: _description_
    """
    
    tsne_decomposer = TSNE(n_components=n_components, 
                           random_state=RS, 
                           n_jobs=-1)
    
    X_tsne = tsne_decomposer.fit_transform(X)
    
    tsne_data = pd.DataFrame(X_tsne, columns=['x', 'y', 'z'])
    tsne_data['label'] = y
    
    fig, ax = plt.subplots(1, 3, figsize=(20, 5))
    palette = 'muted'
    
    sns.scatterplot(tsne_data, 
                    x='x', y='y', hue='label', 
                    palette=palette, 
                    ax=ax[0])
    sns.scatterplot(tsne_data, 
                    x='y', y='z', hue='label', 
                    palette=palette, 
                    ax=ax[1])
    sns.scatterplot(tsne_data, 
                    x='x', y='z', hue='label', 
                    palette=palette, 
                    ax=ax[2])

    fig.suptitle(title)
    fig.show()
    
    data['cluster'] = y

    for i in np.unique(y):
        plot_samples_images(i, data)

#### 2. Настройка параметров алгоритма 

In [23]:
def tune_one_param(X, 
                   model, 
                   p_name, 
                   p_space):
    
    space_list = []
    chs_list = []
    dbs_list = []

    for param in p_space:
        model.set_params(**{p_name: param})
        y = model.fit_predict(X)
        
        if len(np.unique(y)) == 1:
            continue # на случай если модель, например DBSCAN, покажет один единственный кластер
        else:
            space_list.append(param)
            chs_list.append(calinski_harabasz_score(X, y))
            dbs_list.append(davies_bouldin_score(X, y))
            
    scores_data = pd.DataFrame({'chs': chs_list, 
                                'dbs': dbs_list}, 
                               index=space_list)
    
    chs_param = (scores_data
                 .sort_values('chs', ascending=False)
                 .index[0])
    dbs_param = (scores_data
                 .sort_values('dbs', ascending=True)
                 .index[0])
    
    return {'Calinski-Harabasz': chs_param, 
            'Davies-Bouldin': dbs_param} 


def iterative_train(data, 
                    model,  
                    cut_size=5000):
    i_from = list(range(0, 
                        data.shape[0], 
                        cut_size))
    i_to = list(range(cut_size, 
                      data.shape[0], 
                      cut_size)) + [data.shape[0]]

    y = []
    
    for i, j in zip(i_from, i_to):
        X = data[i:j]
        model.fit(X)
        
        [y.append(label) for label in model.labels_]
    
    return pd.Series(y) 


def show_opt_clusters(X, 
                      title, 
                      model, 
                      p_name, 
                      p_space):
    
    X_sample = get_sample(X)
    scores_params = tune_one_param(X_sample, 
                                   model, 
                                   p_name, 
                                   p_space)
    
    print(f"Got optimal '{p_name}' values:\n{scores_params}\n")
    
    for score, param in scores_params.items():
        model.set_params(**{p_name: param})
        y = model.fit_tredict(X_sample)
        
        n_clusters = len(np.unique(y))
        print(f'Got {n_clusters} clusters according to {score} score with model:\n{model}\n')

        clusters_visualiser(X, y, title+f' ({score})')

In [ ]:
eps_space = (np.linspace(0.01, 100, 100, 
                         dtype=float)
             .round(2)
             .tolist())

    
opt_dbscan = lambda X, title: show_opt_clusters(X, 
                                                title, 
                                                model=DBSCAN(n_jobs=-1), 
                                                p_name='eps', 
                                                p_space=eps_space)

#### 3. Визуализация полученных кластеров и их изображений

In [ ]:
opt_dbscan(X_effnet, effnet_name) 

In [ ]:
opt_dbscan(X_osnet, osnet_name)

In [ ]:
opt_dbscan(X_vdc_color, vdc_color_name) 

In [ ]:
opt_dbscan(X_vdc_type, vdc_type_name) 

## 3.1 Метод MiniBatchKMeans

Поскольку исходных данных много, могут возникнуть проблемы с оперативной памятью и скоростью работы, например при использовании классического алгоритма `K-means`, можно воспользоваться реализацией `MiniBatchKMeans`. 

In [ ]:
def get_clustering_scores(model, data, c_range, rs=None):
    chs_list = []
    dbs_list = []
    
    for c in c_range:
        
        if rs is not None:
            model_ = model(c, random_state=RS)    
        else: 
            model_ = model(c)
        
        labels = model_.fit_predict(data)
        
        chs_list.append(calinski_harabasz_score(data, labels))
        dbs_list.append(davies_bouldin_score(data, labels))
    
    return chs_list, dbs_list


def get_n_clusters(df, aggf):
    c_dict = {}
    
    for col in df.columns:
        if aggf == 'max':
            c_dict[col] = (df[col]
                           .sort_values(ascending=False)
                           .index[0])
        if aggf == 'min':
            c_dict[col] = (df[col]
                           .sort_values(ascending=True)
                           .index[0])
        
    return c_dict

In [ ]:
datasets = {'efficientnet': X_effnet, 
            'osnet': X_osnet, 
            'vdc_color_reg': X_vdc_color, 
            'vdc_type_cl': X_vdc_type}

clusters = list(range(2, 11))


def show_clusters_and_scores(model,
                             datasets=datasets, 
                             clusters=clusters, 
                             rs=None, 
                             sample_data=False):
    
    chs_df = pd.DataFrame(index=clusters)
    chs_name = 'Calinski-Harabasz'
    
    dbs_df = pd.DataFrame(index=clusters) 
    dbs_name = 'Davies-Bouldin'
    
    for name, data in datasets.items():
        if sample_data:
            data = get_sample(data)
        
        if rs is not None:
            chs_df[name] = get_clustering_scores(model, data, clusters, rs)[0]
            dbs_df[name] = get_clustering_scores(model, data, clusters, rs)[1]
        else: 
            chs_df[name] = get_clustering_scores(model, data, clusters)[0]
            dbs_df[name] = get_clustering_scores(model, data, clusters)[1]
    
    metsics = pd.DataFrame(data=[get_n_clusters(chs_df, 'max'), 
                                 get_n_clusters(dbs_df, 'min')], 
                           index=[chs_name, dbs_name])
    
    return metsics


show_clusters_and_scores(MiniBatchKMeans, rs=RS)

In [ ]:
additional_message = ' labels and their counts'

print_series = lambda name, data: print(f'{name}{additional_message}:\n {data}', '\n', 
                                        pd.Series(data).value_counts().to_dict(), '\n')


def train_minibatchkmeans(X, n):
    mbkm = MiniBatchKMeans(n_clusters=n, random_state=RS)
    y = mbkm.fit_predict(X)
    
    return y


def n_visualiser(X, n_clusters, train_func, title, n_components=3):
    y = train_func(X, n_clusters)
    print_series(title, y)
    
    clusters_visualiser(X, y, title, n_components)

### Интерпретация кластеров

In [ ]:
clusters_visualiser(X_effnet, y_effnet, effnet_name) 

*Разница между кластерами очень слабая, так как на графике точки очень плотно расположены друг к другу и образуют четкий эллипс в его центре.*

In [ ]:
clusters_visualiser(X_osnet, y_osnet, osnet_name) 

0. прочие
1. белые сзади
2. черные спереди
3. белые спереди
4. черные сзади
5. серебристые спереди
6. прочие (на фото есть желтый цвет)
7. синие
8. красные 
9. зеленые

In [ ]:
clusters_visualiser(X_vdc_color, y_vdc_color, vdc_color_name)

1. серебристые
2. белые 
3. черные 
4. цветные

In [ ]:
clusters_visualiser(X_vdc_type, y_vdc_type, vdc_type_name) 

1. спортивные?
2. грузовые и внедорожники
3. легковые

### Метод Аггломеративной кластеризации

In [ ]:
show_clusters_and_scores(AgglomerativeClustering, sample_data=True) 

In [ ]:
def train_agglomerative_clustering(data, n_clusters, cut_size=5000):
    i_from = list(range(0, 
                        data.shape[0], 
                        cut_size))
    i_to = list(range(cut_size, 
                      data.shape[0], 
                      cut_size)) + [data.shape[0]]

    y = []
    
    for i, j in zip(i_from, i_to):
        X = data[i:j]
        
        model = AgglomerativeClustering(n_clusters=n_clusters)
        y_ = model.fit_predict(X).tolist()
        
        [y.append(label) for label in y_]
    
    return np.array(y) 

In [ ]:
y_effnet = train_agglomerative_clustering(X_effnet, 2)
print_series(effnet_name, y_effnet)

clusters_visualiser(X_effnet, y_effnet, effnet_name) 

In [ ]:
y_osnet = train_agglomerative_clustering(X_osnet, 9)
print_series(osnet_name, y_osnet)

clusters_visualiser(X_osnet, y_osnet, osnet_name) 

In [ ]:
y_vdc_color = train_agglomerative_clustering(X_vdc_color, 3) 
print_series(vdc_color_name, y_vdc_color)

clusters_visualiser(X_vdc_color, y_vdc_color, vdc_color_name)

In [ ]:
y_vdc_type = train_agglomerative_clustering(X_vdc_type, 3)
print_series(vdc_type_name, y_vdc_type) 

clusters_visualiser(X_vdc_type, y_vdc_type, vdc_type_name) 

*Аггломеративная кластеризация не показала достаточной эффективности*

## Метод Гауссовой смеси

In [ ]:
show_clusters_and_scores(GaussianMixture, rs=RS, sample_data=True) 

In [ ]:
def train_gaussianmixture(X, n):
    model = GaussianMixture(n_components=n, random_state=RS)
    y = model.fit_predict(X)
    
    return y 

In [ ]:
y_effnet = train_gaussianmixture(X_effnet, 2)
print_series(effnet_name, y_effnet)

clusters_visualiser(X_effnet, y_effnet, effnet_name) 

In [ ]:
y_osnet = train_gaussianmixture(X_osnet, 7)
print_series(osnet_name, y_osnet)

clusters_visualiser(X_osnet, y_osnet, osnet_name)

In [ ]:
y_vdc_color = train_gaussianmixture(X_vdc_color, 2)
print_series(vdc_color_name, y_vdc_color)

clusters_visualiser(X_vdc_color, y_vdc_color, vdc_color_name)

In [ ]:
y_vdc_type = train_gaussianmixture(X_vdc_type, 5)
print_series(vdc_type_name, y_vdc_type) 

clusters_visualiser(X_vdc_type, y_vdc_type, vdc_type_name)